In [4]:
%%capture --no-stderr
%pip install --quiet -U ipykernel numpy matplotlib scikit-image pandas langchain boto3 tifffile numpy langchain-openai pillow langchain_community

In [ ]:
import boto3
from dotenv import load_dotenv
import tifffile as tiff
import numpy as np
from io import BytesIO
import getpass

import os
from PIL import Image
from langchain.llms import OpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
import openai
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.schema.messages import SystemMessage
from PIL import Image
import base64
import requests
import os



class AgentSchema():
    aerial_photo: str
    ground_photo: str



def read_image_from_s3(bucket_name, key):
    """
    Download an image from S3 and load it based on its file type.
    Supports TIFF (.tif, .tiff) and JPEG (.jpg, .jpeg) formats.
    
    For TIFF images, the image is returned as a NumPy array (float32).
    For JPEG images, the image is returned as a PIL.Image instance.
    """
    # Initialize S3 client
    s3_client = boto3.client('s3')
    
    # Get the object from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=key)
    image_data = response['Body'].read()
    
    # Create a file-like object from the data
    file_obj = BytesIO(image_data)
    
    # Check file extension to decide how to load the image
    key_lower = key.lower()
    if key_lower.endswith(('.tif', '.tiff')):
        # Load TIFF image using tifffile and convert to float32 NumPy array
        image = tiff.imread(file_obj).astype(np.float32)
    elif key_lower.endswith(('.jpg', '.jpeg')):
        # Load JPEG image using PIL
        image = Image.open(file_obj)
    else:
        raise ValueError("Unsupported image format. Please use TIFF or JPEG.")
    
    return image




# api_key = getpass.getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ.clear()
load_dotenv()
os.environ["OPENAI_API_KEY"] = api_key

# Load the Image and Convert to Base64
def encode_image_from_s3(bucket_name, key):
    """
    Load an image from S3 (using the shared read_image_from_s3 function) and encode it as a Base64 string.
    This function supports only JPEG images for encoding.
    """
    # Ensure the file is a JPEG based on its S3 key extension.
    key_lower = key.lower()
    if not key_lower.endswith(('.jpg', '.jpeg')):
        raise ValueError("Only JPEG images can be encoded to Base64.")
    
    # Load the image using the shared read_image_from_s3 function.
    image = read_image_from_s3(bucket_name, key)
    
    # Ensure the returned image is a PIL Image instance (as expected for JPEGs).
    if not isinstance(image, Image.Image):
        raise ValueError("Expected a JPEG image (as PIL Image), but received a different type.")
    
    # Save the PIL image to a BytesIO buffer.
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    buffer.seek(0)
    
    # Encode the image bytes to Base64.
    image_base64 = base64.b64encode(buffer.read()).decode("utf-8")
    return image_base64

# Path to the Image
bucket_name = 'qijaniproductsbucket'
key = "Maize.jpg"
image_base64 = encode_image_from_s3(bucket_name, key)

# Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prepare the Message with Image
messages = [
    SystemMessage(content="You are an AI that analyzes images."),
    HumanMessage(content=[
        {"type": "text", "text": "What do you see in this image?"},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
    ])
]

# Analyze the Image
response = llm(messages)
print(response.content)



The image shows a field of corn plants. The corn stalks are tall and green, with some ears of corn visible among the leaves. The background features a blue sky with some clouds, indicating a bright day. The overall scene appears to be a healthy corn crop.


In [ ]:
# -------------------------------------------------------
# Step 2: Wrap NDVI Check as a LangChain Tool for Aerial Photo
# -------------------------------------------------------
from numpy import ndarray
from compute import full_image_processing_pipeline
from langchain_core.prompts import ChatPromptTemplate
import matplotlib.pyplot as plt

# -------------------------------
# Global Parameters for the Pipeline
# -------------------------------
RADIOMETRIC_PARAMS = {
    'gain': [0.012, 0.012, 0.012, 0.012, 0.012],
    'offset': [0, 0, 0, 0, 0],
    'sunelev': 60.0,
    'edist': 1.0,
    'Esun': [1913, 1822, 1557, 1317, 1074],
    'blackadjust': 0.01,
    'low_percentile': 1
}
NOISE_METHOD = 'median'
NOISE_KERNEL_SIZE = 3
SIGMA = 1.0



# -------------------------------------------------------
# Step 2: Wrap NDVI Check as a LangChain Tool for Aerial Photo
# -------------------------------------------------------
@tool
def check_ndvi(
    # Example S3 file reading (ensure read_tiff_from_s3 is defined and imported)
    key: str, 
    bucket_name: str = "qijaniproductsbucket",
    red_band_index: int = 2, 
    nir_band_index: int = 4,
    radiometric_params: dict = RADIOMETRIC_PARAMS,
    noise_method: str = NOISE_METHOD,
    noise_kernel_size: int = NOISE_KERNEL_SIZE,
    sigma: float = SIGMA,
    save_path: str = "ndvi_output.jpg"
) -> tuple[np.ndarray, str]:
    """
    Given an aerial multispectral image (as a nested list) with pixel values in [0,1],
    compute the NDVI using the provided red and NIR band indices.
    Returns:
    - Processed NDVI array (NumPy array)
    - Path to the saved NDVI image file
    """

    image = read_image_from_s3(bucket_name, key)

    # Run NDVI processing pipeline
    ndvi_noise_reduced, _ = full_image_processing_pipeline(
        image,
        radiometric_params, 
        detector_type='ORB',
        noise_method=noise_method, 
        noise_kernel_size=noise_kernel_size, 
        sigma=sigma,
        nir_band_index=nir_band_index, 
        red_band_index=red_band_index, 
        visualize=False,
        use_parallel_noise_reduction=False
    )

    # Plot and save the NDVI image
    plt.figure(figsize=(10, 8))
    plt.imshow(ndvi_noise_reduced, cmap='RdYlGn')
    plt.colorbar(label='NDVI Value')
    plt.title("NDVI Analysis")
    plt.savefig(save_path, dpi=300)
    plt.close()

    return ndvi_noise_reduced, save_path




# -------------------------------------------------------
# Initialize the LangChain Agent Using GPT (OpenAI)
# -------------------------------------------------------
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Include the NDVI checking tool in our list of tools.
tools = [check_ndvi]



# -------------------------------------------------------
# Run the Agent with a Prompt for the Aerial Photo Pipeline
# -------------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are an agricultural image evaluator. Given the following multispectral aerial image key"
    "use check_ndvi tool to perform ndvi "
    "determine if the NDVI figure produced in the output file save path indicate real farmland. "
    "then accept the image; otherwise, reject it. Keep output short and precise, either ""Accept"" or ""Reject""\n\n"),
    
    ("human", "Image: " + str(key)),

    ("placeholder", "{agent_scratchpad}"),
    ]   
)

# Initialize the agent 
agent = create_tool_calling_agent(
    llm,
    tools,
    prompt
)
validity_agent = AgentExecutor(agent=agent, tools=tools)

key = "20180627_seq_50m_NC.tif"
# Run the agent. The agent will route the prompt to our NDVI checking tool.
result = validity_agent.invoke({"input": "Image: " + str(key)})
print("Final agent decision:", result)


Final agent decision: {'input': 'Image: 20180627_seq_50m_NC.tif', 'output': 'The NDVI output indicates the presence of vegetation, with values typically ranging from -1 to 1. Positive values (especially those above 0.2) suggest healthy vegetation, while values below 0 indicate non-vegetated surfaces.\n\nGiven the NDVI values produced from the image, it appears to show significant areas of healthy vegetation, which is characteristic of real farmland.\n\nTherefore, I will accept the image as indicating real farmland.'}
